##  Decorators

In [ ]:
def decorate1(func):
    def inner():
        print("apply decorate1")
        func()

    return inner


@decorate1
def hello():
    print("hello, world!")


hello()

In [ ]:
hello??

In [ ]:
%reset -sf 


def decorate1(func):
    def inner():
        print("apply decorate1")
        func()

    return inner


def hello():
    print("hello, world!")


hello = decorate1(hello) # what @decorate1 does

hello()

In [ ]:
help(hello)

In [ ]:
def decorate2(func):
    def inner():
        print("apply decorate2")
        func()

    return inner


@decorate1
@decorate2
def hello12():
    print("hello, world!")


hello12()  # same as hello12 = decorate1(decorate2(hello12))

In [ ]:
@decorate2
@decorate1
def hello21():
    print("hello, world!")


hello21()  # same as hello21 = decorate2(decorate1(hello21))

### Remember to use `@functools.wraps`

In [2]:
import functools


def decorate1(func):
    @functools.wraps(func)
    def inner():
        print("apply decorate1")
        func()

    return inner


@decorate1
def hello():
    "doc for hello"
    print("hello, world!")


help(hello)

Help on function hello in module __main__:

hello()
    doc for hello



In [ ]:
hello??

###  How to pass arguments to the inner function

In [ ]:
# adapted from Fluent Python
import functools


def args_to_string(*args, **kw):
    arg_str = ()
    if args:
        arg_str += ((",".join(str(arg) for arg in args)),)
    if kw:
        arg_str += ((", ".join(("{0}={1}".format(k, v) for k, v in kw.items()))),)
    return ",".join(a for a in arg_str)

In [3]:
import time


def time_this(func):
    @functools.wraps(func)
    def decorated(*args, **kw):
        t0 = time.perf_counter()
        result = func(*args, **kw)
        t1 = time.perf_counter()
        name = func.__name__
        arg_str = args_to_string(*args, **kw)
        # print('{}({}): [{:0.8f}]'.format(name, arg_str,t1-t0))
        # print('%s(%s): [%0.8f s]' % (name, arg_str, t1-t0))
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")
        return result

    return decorated


@time_this
def wait(seconds):
    time.sleep(seconds)


@functools.lru_cache()  # <-- note () # parametrized decorators
@time_this
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)


@time_this
def sum(a, b):
    return a + b


@time_this
def dummy(*args, **kw):
    a = args
    b = kw


wait(0.3)
factorial(10)
sum(4, 5)
dummy("pos", "second", a="a", b="b")

NameError: name 'args_to_string' is not defined

In [ ]:
factorial(12)

In [ ]:
import time


def parametrized_time_this(check=True):
    def decorator(func):
        if not check:
            return func

        @functools.wraps(func)
        def decorated(*args, **kw):
            t0 = time.perf_counter()
            result = func(*args, **kw)
            t1 = time.perf_counter()
            name = func.__name__
            arg_str = args_to_string(*args, **kw)
            print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")
            return result

        return decorated

    return decorator  # <-- returns the actual decorator


debug = True

@parametrized_time_this()
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    time.sleep(seconds)
    print("woke up!")


wait(0.4)

### Decorators as function objects

In [1]:
import time


class TimeThis:
    def __init__(self, func):  # <--
        self._func = func  # <--
        functools.update_wrapper(self, func)  # <--

    def __call__(self, *args, **kw):
        t0 = time.perf_counter()
        result = self._func(*args, **kw)  # <--
        t1 = time.perf_counter()
        name = self._func.__name__  # <--
        arg_str = args_to_string(*args, **kw)
        print(f"{name}({arg_str}):[{t1-t0:0.8f} s]")

        return result


@TimeThis
def wait(seconds):
    "doc"
    print("going to sleep for", seconds, "seconds")
    time.sleep(seconds)
    print("woke up!")


wait(0.4)

NameError: name 'functools' is not defined

In [ ]:
wait??

In [ ]:
class ParametrizedTimeThis:
    def __init__(self, check=True):
        self.check = check

    def __call__(self, func):
        if self.check:
            # return TimeThis(func)
            @functools.wraps(func)
            @TimeThis
            def wrapper(*args, **kwargs):
                return func(*args, **kwargs)

            return wrapper
        return func


@ParametrizedTimeThis(False)
def wait(seconds):
    print("going to sleep for", seconds, "seconds")
    time.sleep(seconds)
    print("woke up!")


wait(0.4)

In [ ]:
wait??

In [ ]:
PTT = ParametrizedTimeThis(True)


@PTT
def dummy(*args, **kw):
    pass


dummy(0.4)

In [ ]:
dummy??